# Goroutines and channels, what and why?

## Imports

In [ ]:
import (
    "fmt"
    "time"
)

## goroutines

A goroutine is a lightweight thread of execution. In other words from Bill Kennedy:

> Goroutines are functions that are created and scheduled to be run independently by the Go scheduler. The Go scheduler is responsible for the management and execution of goroutines.

These goroutines aren't the same as OS "threads." Threads are expensive OS operations. You run out of system resources very quickly when launching threads. Goroutines are more like a "coroutine" or "coprogram." These coroutines multiplex independently executing functions onto a set of threads. 

You can launch a new goroutine by simply using `go f(s)`, as follows:

In [ ]:
// Define a function that prints out numbers.
func printNum(from string) {
    for i := 1; i < 5; i++ {
        fmt.Printf("%s output: %d\n", from, i)
        time.Sleep(1*time.Second)
    }
} 

In [ ]:
// Start a goroutine that will print some numbers.
go printNum("goroutine 1")

// Start another goroutine that will print some numbers.
go func() {
    fmt.Println("goroutine 2 output!")
}()

time.Sleep(4*time.Second)

## Channels

Go channels facilitate communication or signaling between goroutines. We will cover some of the basic patterns of channel usage here, but there are a whole variety of types and patterns for Go channels. I highly recommend that you read [this blog post](https://www.goinggo.net/2017/10/the-behavior-of-channels.html) for a detailed introduction. 

In [ ]:
// Make a channel that will signal with string data.
ch := make(chan string)

In [ ]:
// Start a goroutine that will print received string messages.
go func() {
    p := <-ch
    fmt.Println("received signal: ", p)
}()

time.Sleep(time.Second)

// Send a message on the channel.
ch <- "This is my message"
fmt.Println("sent signal")

time.Sleep(time.Second)

### Buffered and Unbuffered Channels

When signaling with some kind of data (e.g., strings as above), we can use a buffered or unbuffered channel. Our choice will have some impacts on whether we can garauntee delivery of the signal. 

- If you need to have a goroutine do something only when it receives a signal (i.e., it needs to wait for a signal), you should use an unbuffered channel because you need to ensure that the signal is received.
- When you need to throw multiple workers at a problem and don't need to ensure receipt of an individual signal, you might use a buffered channel with a well defined number of workers. You might also use a buffered channel when you need to drop signals after being saturated with a well defined number of signals.

The example included above in the "Channels" section is an example of of the first case, when a goroutine is waiting on a signal. Once the goroutine receives the signal, it does the corresponding work (printing the message in this case).

Here is an example of the second case, when we are fanning out, such that multiple workers can help us accomplish a task:

In [ ]:
// Create our buffered channel.
workers := 20
results := make(chan string, workers)

In [ ]:
// Have our workers complete some expensive tasks.
for w := 0; w < workers; w++ {
    output := fmt.Sprintf("Worker %d output", w)
    go func(output string) {
        time.Sleep(1*time.Second)
        results <- output
    }(output)
}

In [ ]:
// Collect the results.
for i := 0; i < workers; i++ {
    p := <-results
    fmt.Println(p)
}

**Note** - If you are going to be using a buffered channel, you should _always_ have some justification for the size of that buffered channel. If you don't know how big to make your channel, you should take a second and think through it. [This article](https://www.goinggo.net/2017/10/the-behavior-of-channels.html) should help with that.